In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Product attributes extraction and detailed descriptions from images using Gemini 1.5 Pro

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retail/product_attributes_extraction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fuse-cases%2Fretail%2Fproduct_attributes_extraction.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/retail/product_attributes_extraction.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/retail/product_attributes_extraction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

| | |
|-|-|
|Author(s) | [Tianli Yu](https://github.com/tianli) |

## Overview

This Colab teaches how to build a general agent (on top of Gemini) to extract different product attributes or detailed product descriptions from an image input. It also introduces a prompting technique called "self-correcting prompt" where you can ask the model to check and verify the result by itself (all in one single prompt). Self-correcting prompt proves to improve the overall quality of the agent's output.

In the following sections we will:

*   Write the necessary image loading and parsing library.
*   Create a product image agent.
*   Run the agent on a set of examples for tasks like product image description and attribute extraction.

## Get started

### Install Vertex AI SDK and other required packages

In [ ]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [1]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
PROJECT_ID = "YOUR_PROJECT_ID"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


import vertexai

vertexai.init(project=PROJECT_ID, location=LOCATION)

## Build a Product Image Agent

In [3]:
# @title Image loading and parsing library
import json

import requests
from vertexai.preview.generative_models import Part


def get_mime_from_uri(image_uri: str) -> str:
    """Get the mime type from the image uri."""
    if image_uri.endswith(".png"):
        return "image/png"
    elif image_uri.endswith(".gif"):
        return "image/gif"
    else:
        # Assume JPEG as the default mime
        return "image/jpeg"


def load_image_part_from_uri(image_uri: str) -> None | Part:
    """Load image to a prompt Part from a remote or local file URI.

    Args:
        image_uri (str): The uri or the local file path for the image.

    Returns:
        The prompt Part object.
    """
    if image_uri.startswith("gs://"):
        return Part.from_uri(image_uri, mime_type=get_mime_from_uri(image_uri))
    elif image_uri.startswith("http://") or image_uri.startswith("https://"):
        response = requests.get(image_uri)
        if response.status_code == 200:
            image_bytes = response.content
            return Part.from_data(image_bytes, mime_type=get_mime_from_uri(image_uri))
        else:
            print(
                f"Fetch image failed for {image_uri}, status code: {response.status_code}"
            )
            return None
    else:
        image_bytes = open(image_uri, "rb").read()
        return Part.from_data(image_bytes, mime_type=get_mime_from_uri(image_uri))


def parse_json_from_markdown(answer: str) -> str:
    """Parse the json from the markdown answer.

    Args:
        answer (str): The markdown answer from the model.

    Returns:
        A parsed json dictionary}
    """
    lines = answer.split("```")
    try:
        # Tries to parse the last json in the answer.
        answer = lines[-2]
        if answer.startswith("json"):
            answer = answer[4:]
        result = json.loads(answer)
    except json.JSONDecodeError:
        # Falls back to the first json in the answer.
        answer = lines[1]
        if answer.startswith("json"):
            answer = answer[4:]
        result = json.loads(answer)
    return json.dumps(result)

In [44]:
# @title The ProductImageAgent Class.


from IPython.display import display
import ipywidgets as widgets
from vertexai.preview.generative_models import GenerationConfig, GenerativeModel, Part


class ProductImageAgent:
    """An agent that wraps around Gemini 1.5 to extract product attributes from
    images.

    Args:
        gemini_model_version (str): The version string of the Gemini 1.5 model.
            gemini-1.5-pro or gemini-1.5-flash
        temperature (float): The temperature of the model. Defaults to 1.0.
        max_output_tokens (int): The maximum number of output tokens. Defaults to
            8192.
    """

    def __init__(
        self,
        gemini_model_version: str = "gemini-1.5-pro",
        temperature: float = 0.0,
        max_output_tokens: int = 8192,
    ):
        config = GenerationConfig(
            temperature=temperature, max_output_tokens=max_output_tokens
        )

        # System instructions, add any common instructions here.
        sys_inst = """
    As an assistant for an online retailer, your task is to recognize
    attributes from the provided product image.
    If an attribute vocabulary is provided, please only select attribute values
    in that vocabulary. You answer should be strictly consistent with what's in
    the image. If any attributes do not exist in the image, please
    return null for that attribute.
    """
        self.gemini_model = GenerativeModel(
            gemini_model_version, generation_config=config, system_instruction=sys_inst
        )

    def get_detailed_description(self, image_uri: str, debug: bool = False) -> str:
        """Generates the detailed product description from an image.

        Args:
            image_uri: The url to the image, can be a local file path,  or a url
            from the web or gcs.

        Returns:
            The generated detailed description from the model response.
        """
        image_part = load_image_part_from_uri(image_uri)
        if image_part is None:
            return ""
        prompt = """
        Please write a complete and detailed product description for the
        above product image. The length of the description should be at least
        200 words.
        """
        if debug:
            print("====== Begin Debug Info ======")
            preview = widgets.HTML(value=f'<img src="{image_uri}" width="512">')
            display(preview)
            print(f"Prompt:\n{prompt}")
            print("====== End Debug Info ======")

        model_response = self.gemini_model.generate_content([image_part, prompt])
        return model_response.text

    def get_attributes(
        self,
        image_uri: str,
        product_category: None | str = None,
        vocabulary_json: None | str = None,
        debug: bool = False,
    ) -> str:
        """Generates the product attributes from an image.

        Args:
            image_uri (str): The uri of the product image to generate attributes.
            vocabulary_json (str): A json string list all the attribute names and
            their possible vocabulary.

        Returns:
            The product attribute json string from the model response.
        """
        image_part = load_image_part_from_uri(image_uri)
        if image_part is None:
            return ""
        if product_category:
            prompt = f"""
          The above image is a product image from the {product_category}
          category.
          Please list all the relevant attributes in the {product_category}
          category for the main product in the above image and return a list of
          key-value pairs in json format.
          """
        else:
            prompt = """
          Please recognize the main product's all relevant attributes in the
          above image and return a list of key-value pairs in json format.
          """
        if vocabulary_json:
            prompt += f"""
          Please using only the vocabulary defined in the following json:
          {vocabulary_json}
          For each key, you should select the most appropriate attribute value
          from its corresponding vocabulary list and returns one value
          for each attribute key.
          You can return null for that key if no attributes match.
          """
        if debug:
            print("====== Begin Debug Info ======")
            preview = widgets.HTML(value=f'<img src="{image_uri}" width="512">')
            display(preview)
            print(f"Prompt:\n{prompt}")
            print("====== End Debug Info ======")

        model_response = self.gemini_model.generate_content([image_part, prompt])
        return model_response.text

    def get_attributes_self_correcting_prompt(
        self,
        image_uri: str,
        product_category: None | str = None,
        vocabulary_json: None | str = None,
        debug: bool = False,
    ) -> str:
        """Generates the product attributes from an image using self-correcting prompt.

        Args:
            image_uri (str): The uri of the product image to generate attributes.
            vocabulary_json (str): A json string list all the attribute names and
            their possible vocabulary.

        Returns:
            The product attribute json string from the parsed model response.
        """
        image_part = load_image_part_from_uri(image_uri)
        if image_part is None:
            return ""
        if product_category:
            prompt = f"""
          The above image is a product image from the {product_category}
          category.
          First please list all the relevant attributes in the
          {product_category} category for the main product in the above
          image and return a list of key-value pairs in json format.
          """
        else:
            prompt = """
          First Please recognize the main product's all relevant attributes in
          the above image and return a list of key-value pairs in json
          format.
          """

        if vocabulary_json:
            prompt += f"""
          Please using only the vocabulary defined in the following json:
          {vocabulary_json}
          For each key, you should select the most appropriate attribute value
          from its corresponding vocabulary list and return the attribute key-
          value pair. You can return null for that key if no attributes match.
          """

        # Adding the self-correction instructions.
        prompt += """
        Next, treat the returned json as the result generated by a different
        model, rate each key-value pair as "correct" or "wrong" based on the
        same image. You can output in a format like "key - value: correct (or
        wrong)".
        Then, based on this evaluation, please update all the attributes that
        are corrected in the final json output.
        Please use markdown to annotate different json in your output.
        """
        model_response = self.gemini_model.generate_content([image_part, prompt])

        if debug:
            print("====== Begin Debug Info ======")
            preview = widgets.HTML(value=f'<img src="{image_uri}" width="512">')
            display(preview)
            print(f"Prompt:\n{prompt}\n")
            print(f"Response:\n{model_response.candidates[0].content.parts[0].text}\n")
            print("====== End Debug Info ======")

        # Parse the model_response to get the final json.
        return parse_json_from_markdown(
            model_response.candidates[0].content.parts[0].text
        )


# Creates the agent.
product_agent = ProductImageAgent(
    gemini_model_version="gemini-1.5-pro-preview-0514", temperature=0
)

In [ ]:
# @title Example 1: Generate detailed product description from an image.
image_uri = "IMAGE_URI"  # @param {type:"string"}

product_agent.get_detailed_description(image_uri, True)

In [ ]:
# @title Example 2: Get product attribute json from an image -- open vocabulary.
image_uri = "IMAGE_URI"  # @param {type:"string"}

# Open vocabulary.
attribute_json = product_agent.get_attributes(image_uri, debug=True)
print(f"Open vocabulary attributes:\n{attribute_json}")

In [ ]:
# @title Example 3: Get product attribute json from an image -- closed vocabulary.
image_uri = "IMAGE_URI"  # @param {type:"string"}

# Closed vocabulary.
vocabulary = """
{"Pattern": ["Animal", "Letter", "Plaid", "Plain", "Polka Dot", "Quilted", "Striped", "Tie Dye", "Tropical", "Zebra", "Block", "Rainbow", "Floral"], "Toe": ["Almond Toe", "Cap Toe", "Closed Toe", "Peep Toe", "Point Toe", "Pointed Toe", "Round Toe", "Square Toe", "Toe Post", "Open Toe"], "Style": ["Ballet", "Bandage", "Basics", "Casual", "Classic", "Cute", "Elegant", "Formal", "Modern", "Motorcycle", "Retro", "Sexy",  "Boho", "Modest", "Comfort", "Minimalist"], "Strap Type": ["Adjustable", "Ankle cuff", "Ankle straps", "Chain", "Convertible", "Criss Cross", "D'orsay", "Double Handle", "Flowers", "Gladiator", "Lace Up", "Mary Jane", "Ring", "Slingbacks", "Strappy", "T strap", "Zipper", "Elastic", "Velcro", "Ankle Strap"], "Heels": ["Chunky", "Cork", "Espadrilles", "Flat", "Flatform", "Platform", "Stiletto", "Cone Heel", "Kitten Heels", "Hidden Wedge", "Wedges", "Pyramid"], "Closure Type": ["Back Zipper", "Buckle", "Zipper", "Magnet", "Slip on", "Hook Loop", "Lace-up", "Flap"]}
"""

attribute_json = product_agent.get_attributes(
    image_uri, vocabulary_json=vocabulary, debug=True
)
print(f"Closed vocabulary attributes\n{attribute_json}")

In [ ]:
# @title Example 4: Get product attribute json from an image using self-correcting prompt -- open vocabulary
image_uri = "IMAGE_URI"  # @param {type:"string"}

# Open vocabulary.
attribute_json = product_agent.get_attributes_self_correcting_prompt(
    image_uri, product_category="Shoes", debug=True
)
print(f"Open vocabulary attributes:\n{attribute_json}")

In [ ]:
# @title Example 5: Get product attribute json from an image using self-correcting prompt -- closed vocabulary
image_uri = "IMAGE_URI"  # @param {type:"string"}

# Closed vocabulary.
vocabulary = """
{"Pattern": ["Animal", "Letter", "Plaid", "Plain", "Polka Dot", "Quilted", "Striped", "Tie Dye", "Tropical", "Zebra", "Block", "Rainbow", "Floral"], "Toe": ["Almond Toe", "Cap Toe", "Closed Toe", "Peep Toe", "Point Toe", "Pointed Toe", "Round Toe", "Square Toe", "Toe Post", "Open Toe"], "Style": ["Ballet", "Bandage", "Basics", "Casual", "Classic", "Cute", "Elegant", "Formal", "Modern", "Motorcycle", "Retro", "Sexy",  "Boho", "Modest", "Comfort", "Minimalist"], "Strap Type": ["Adjustable", "Ankle cuff", "Ankle straps", "Chain", "Convertible", "Criss Cross", "D'orsay", "Double Handle", "Flowers", "Gladiator", "Lace Up", "Mary Jane", "Ring", "Slingbacks", "Strappy", "T strap", "Zipper", "Elastic", "Velcro", "Ankle Strap"], "Heels": ["Chunky", "Cork", "Espadrilles", "Flat", "Flatform", "Platform", "Stiletto", "Cone Heel", "Kitten Heels", "Hidden Wedge", "Wedges", "Pyramid"], "Closure Type": ["Back Zipper", "Buckle", "Zipper", "Magnet", "Slip on", "Hook Loop", "Lace-up", "Flap"]}
"""
attribute_json = product_agent.get_attributes_self_correcting_prompt(
    image_uri, vocabulary_json=vocabulary, debug=True
)
print(f"Closed vocabulary attributes\n{attribute_json}")